In [1]:
# Matching genes that are knocked out to the type of metabolism they are part of to ascribe them to a specific niche
# - Author: Shauny Van Hoye
# - Date: 2022-05-07

In [2]:
!pip install cobra

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.8/546.8 KB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 KB 4.6 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.13.0
    Uninstalling h11-0.13.0:
      Successfully uninstalled h11-0.13.0


In [3]:
import re
import cobra
import numpy as np
import time

In [4]:
def metabolism(gene):

    # Ecocyc

    ecocyc = open("../input/mt-data/ecocyc.gaf")

    go_codes = []

    for line in ecocyc:
            
        l = line.strip()

        if re.search(gene, l):

            s = re.search('GO:([0-9]{7})', l)

            go_codes.append(s.group(1))

            
    # GO

    key = 0

    sugar = 0

    carbohydrate = 0

    nucleotide = 0

    lipid = 0

    AA = 0

    for code in go_codes:

        GO = open("../input/mt-data/go.obo")

        for line in GO:
                
            l = line.strip()

            if re.match('^id:',l):

                if re.search(code, l):
                    
                    key = 1

                else:

                    key = 0

            if key == 1:

                #print(l)

                if re.search('sugar', l):

                    sugar += 1
                    
                if re.search('carbohydrate', l):

                    carbohydrate += 1

                if re.search('nucleotide', l):
                        
                    nucleotide += 1   

                if re.search('lipid', l):
                            
                    lipid += 1 

                if re.search('amino acid', l):
                        
                    AA += 1         


    return sugar, carbohydrate, nucleotide, lipid, AA

In [5]:
# Test 1

# t0 = time.time()

# genes = ['b1101', 'b1603', 'b3612', 'b3956', 'b3919']

# for gene in genes:

    # sugar, carbohydrate, nucleotide, lipid, AA = metabolism(gene)

    # print(sugar, carbohydrate, nucleotide, lipid, AA)

# t1 = time.time()

# total = t1-t0

# print(total)

In [6]:
# The next code block goes over all the genes in the iML1515 model and makes use of the ecocyc and 
# go database to determine which type of metabolism each gene is correlated with. This code only has to 
# be run once, afterwards, the results are saved in separate files which are in turn used to determine the 
# niches in the Map-Elites algorithm.

# Load the iML1515 E. coli GEM

e_coli_iML1515 = cobra.io.read_sbml_model("../input/mt-data/iML1515.xml")

number_of_genes = len(e_coli_iML1515.genes)

print(number_of_genes)

sugar = np.zeros(number_of_genes) 

carbohydrate = np.zeros(number_of_genes) 

nucleotide = np.zeros(number_of_genes) 

lipid = np.zeros(number_of_genes) 

AA = np.zeros(number_of_genes) 

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
1516


In [7]:
for i in range(0,number_of_genes):

    gene = e_coli_iML1515.genes[i]
    
    sugar[i], carbohydrate[i], nucleotide[i], lipid[i], AA[i] = metabolism(str(gene))


np.save("./sugar_iML1515", sugar)

np.save("./carbohydrate_iML1515", carbohydrate)

np.save("./nucleotide_iML1515", nucleotide)

np.save("./lipid_iML1515", lipid)

np.save("./AA_iML1515", AA)